pip install openpyxl   
pip install xlrd

In [1]:
import numpy as np
import pandas as pd
import requests,json,os
from urllib.parse import quote 

In [2]:
df = pd.read_excel('../../전체_도시철도역사정보_20230317.xlsx',engine='openpyxl')
df.head()

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
0,0101,판암,S3001,대전 도시철도 1호선,Panam,板岩,일반역,NaN,NaN,36.316896,127.458264,대전교통공사,대전광역시 동구 옥천로 지하162 (판암동),042-539-3601,2022-05-09 00:00:00
1,0102,신흥,S3001,대전 도시철도 1호선,Sinheung,新興,일반역,NaN,NaN,36.319652,127.448792,대전교통공사,대전광역시 동구 옥천로 지하 70 (판암동),042-539-3602,2022-05-09 00:00:00
2,0103,대동,S3001,대전 도시철도 1호선,Dae-dong,大洞,일반역,NaN,NaN,36.329532,127.442848,대전교통공사,대전광역시 동구 계족로 지하 176 (대동),042-539-3603,2022-05-09 00:00:00
3,0104,대전,S3001,대전 도시철도 1호선,Daejeon,大田,일반역,NaN,NaN,36.331583,127.433118,대전교통공사,대전광역시 동구 중앙로 지하 218 (중동),042-539-3604,2022-05-09 00:00:00
4,0105,중앙로,S3001,대전 도시철도 1호선,Jungangno,中央路,일반역,NaN,NaN,36.328664,127.425842,대전교통공사,대전광역시 중구 중앙로 지하 145 (은행동),042-539-3605,2022-05-09 00:00:00


In [3]:
df.to_csv('./total_info.csv',index=False)

In [4]:
df = pd.read_csv('./total_info.csv')
df = df[['역사명','역사도로명주소','운영기관명']]
df.rename(columns={'역사명':'지하철역','역사도로명주소':'도로명주소'},inplace=True)
df.운영기관명.unique()
exclude_list = ['대전교통공사', '대구도시철도공사', '부산광역시 부산교통공사', '부산-김해경전철㈜','광주광역시 도시철도공사']
for exclude in exclude_list:
    df = df[df['운영기관명'] != exclude]
df.head()
df.drop(columns=['운영기관명'],inplace=True)
df.shape

(782, 2)

In [5]:
df.to_csv('test_local_info.csv',index=False)

In [6]:
df.reset_index(drop=True,inplace=True)
df.nunique()

지하철역     702
도로명주소    738
dtype: int64

In [7]:
df.head()

,지하철역,도로명주소
0,계양,인천광역시 계양구 다남로 24
1,귤현,인천광역시 계양구 장제로 1136
2,박촌,인천광역시 계양구 장제로 992
3,임학,인천광역시 계양구 장제로 875
4,계산,인천광역시 계양구 경명대로 1089


In [8]:
df['지하철역'][1][:-1]

'귤'

In [9]:
df['지하철역'] = df['지하철역'].str.replace('(', ' ',regex=False,).str.split().str[0]
for i in df.index:
    if df['지하철역'][i][-1] == '역':
        df['지하철역'][i] = df['지하철역'][i][:-1]
df.drop_duplicates(subset=['지하철역'],keep='first',inplace=True)
df.head()

,지하철역,도로명주소
0,계양,인천광역시 계양구 다남로 24
1,귤현,인천광역시 계양구 장제로 1136
2,박촌,인천광역시 계양구 장제로 992
3,임학,인천광역시 계양구 장제로 875
4,계산,인천광역시 계양구 경명대로 1089


In [10]:
df.nunique()

지하철역     656
도로명주소    648
dtype: int64

In [11]:
####################################################################
# 위까지는 전처리
# 아래부터는 도로명주소 기반으로 좌표 구해서 추가함
####################################################################

In [12]:
df.to_csv('test_local_info.csv',index=False)
# 하고 나서 영종 -> 강조문자 삭제
# 도로명주소에 오타, 잘못된 주소 기입된 역명들 수정
# local_info.csv가 수정본

In [69]:
# 1
# 수정본 파일로 데이터프레임 만듬
df = pd.read_csv('local_info_2.csv')

In [70]:
df.head()

,지하철역,도로명주소
0,계양,인천광역시 계양구 다남로 24
1,귤현,인천광역시 계양구 장제로 1136
2,박촌,인천광역시 계양구 장제로 992
3,임학,인천광역시 계양구 장제로 875
4,계산,인천광역시 계양구 경명대로 1089


In [2]:
# 2 kakao API로 도로명 주소 구하는 함수
def kakao_location(place):
    with open('data/main/key/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [3]:
# 2 입력받은 데이터 프레임의 인덱스로 도로명주소 추출하는 함수
def rtn_addr(target):
    str_addr = df[df.지하철역 == target].도로명주소.values[-1]
    return str_addr.strip()

In [4]:
# 3 2번 함수들 사용해서 좌표추가
temp1 =[]
for i in df.index:
    try:
        target = df['지하철역'][i].strip()
        temp1.append(kakao_location(rtn_addr(target)))
    except:
        print(i, df.지하철역[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df2 = pd.concat([df, df_test], axis=1)

NameError: name 'df' is not defined

In [57]:
# 4 처리 끝난 파일 저장
df2.to_csv('stn_r_addr_final.csv',index=False)

In [58]:
# 위 과정에서 에러 발생한 부분 검증
temp = df['지하철역'][430]
temp

'성환'

In [59]:
# 결과적으론 이 과정에서는 에러 발생안함
rtn_addr(temp)

'충청남도 천안시 서북구 성환읍 성환1로 237-5 (성환리 449-490번지)'

In [7]:
# 공공데이터에서 제공한 데이터들에 오타/ 잘못된 주소 기입이 에러의 원인
# 에러가 발생한 역의 도로명 주소 새로 구해서 테스트
kakao_location('경기도 광명시 광명7동')

(37.4693741601211, 126.853066142144)

In [61]:
# 전처리 과정에서 누락된 부분 제거
df3 = df[df['도로명주소'].str.contains('부산|울산')]
df3

,지하철역,도로명주소


In [62]:
df[df.지하철역.values == '태화강']

,지하철역,도로명주소


In [63]:
df = df[~df['도로명주소'].str.contains('부산|울산')]

In [64]:
df.shape

(649, 2)

In [65]:
df[df.지하철역.values == '태화강']

,지하철역,도로명주소


In [66]:
df.shape

(649, 2)

In [67]:
# 누락된 부분 제거 하는 코드
df = df[~df['도로명주소'].str.contains('부산|울산')]

In [ ]:
df.shape

(635, 2)

In [68]:
# 4 처리 끝난 파일 저장
df2.to_csv('stn_r_addr_final.csv',index=False)

In [ ]:
df.to_csv('local_info_2.csv',index=False)